# Example usage of SeisSrcMoment to calculate $M_W$ in the frequency domain, using a linearised $M_0$ method

This example is for a volcano-tectonic earthquake at Uturuncu, Bolivia. The moment magnitude, $M_W$, is calculated in the frequency domain, i.e. the long period spectral level is calculated by fitting a Brune model, as detailed in Stork et al (2014). However, here we use a linearised Brune model to calculate a number of the parameters in the Brune model with enhanced numerical stability over the non-linear Brune fit method. This earthquake's moment tensor is analysed in Alvizuri and Tape (2016), with $M_W$ = 2.80 found by full waveform moment tensor inversion.

## 1. Specify parameters to use:

In [7]:
import numpy as np
import os, sys 
%load_ext autoreload
%autoreload 2
from SeisSrcMoment import moment_linear_reg
from NonLinLocPy import read_nonlinloc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Specify variables:
inventory_fname = "data/instrument_gain_data/IRISDMC-Plutons_dataless.dataless"  # The inventory fname, pointing to the dataless file for the network (for full instrument frequency response removal)
# mseed_filename = "data/mseed_data/20100516063454720000.m" # Note: One can pass the script an obspy stream instead if one wishes.
# NLLoc_event_hyp_filename = "data/NLLoc_data/loc.Tom_RunNLLoc000.20100516.063457.grid0.loc.hyp"
stations_not_to_process = []
window_before_after = [0.1, 0.6] # The time before and after the phase pick to use for calculating the magnitude within
filt_freqs = [0.5, 49.0] # Filter frequencies to apply (important if not removing long period spectral noise)
MT_six_tensors = [] # If this is not specified, assumes average DC component in P (or S) from Stork et al (2014).
density = 2750. #2000. # Density of medium, in kg/m3
Vp = 5000. # P-wave velocity in m/s
# Note that Q not required as the program calculates Q when fitting the source model.
verbosity_level = 0 # Verbosity level (1 for moment only) (2 for major parameters) (3 for plotting of traces)
plot_switch = True
remove_noise_spectrum = False # If True, removes noise using spectrum taken from window before trace. Not thoroughly tested yet, but can get around by applying a high pass filter above anyway.
invert_for_geom_spreading = False
freq_inv_intervals = None #np.arange(0.5, 40, 4.0) #None # Frequency intervals to sample the displacement spectra for the inversion.


One also needs to set the inversion method option for the moment parameters to invert for. The options are as follows:

![title](inversion_options_diagram.png)


In [9]:
# Set inversion method option:
inv_option = "method-3" # Options are: "method-1"; "method-2"; "method-3"; "method-4", as above.

In [10]:
# And simulate for multiple events (TESTING!!!):
mseed_fnames_tmp = np.loadtxt("data/mseed_fnames_list.txt", dtype=str)
nlloc_fnames_tmp = np.loadtxt("data/nlloc_fnames_list.txt", dtype=str)
mseed_filenames = []
NLLoc_event_hyp_filenames = []
for i in range(len(mseed_fnames_tmp)):
    mseed_filenames.append(os.path.join("data/mseed_data", mseed_fnames_tmp[i]))
    NLLoc_event_hyp_filenames.append(os.path.join("data/NLLoc_data", nlloc_fnames_tmp[i]))


## Run moment calculation:

In [16]:
# Find seismic moment release:
moment_linear_reg.calc_moment_via_linear_reg(mseed_filenames, NLLoc_event_hyp_filenames, density, Vp, inventory_fname=inventory_fname, window_before_after=window_before_after, filt_freqs=filt_freqs, stations_not_to_process=stations_not_to_process, MT_six_tensors=MT_six_tensors, verbosity_level=verbosity_level, invert_for_geom_spreading=invert_for_geom_spreading, freq_inv_intervals=freq_inv_intervals, inv_option=inv_option, plot_switch=plot_switch)
# print("Seismic moment release (Nm):", av_M_0)

Skipping station as spectra of length less than usual.
Skipping station as spectra of length less than usual.
Insufficient observations for spectral ratios, therefore skipping event data/NLLoc_data/loc.Tom_RunNLLoc000.20110619.211536.grid0.loc.hyp
Skipping station as spectra of length less than usual.
Skipping station pair 0 - 10 as doesn't exist for current event
Skipping station pair 1 - 10 as doesn't exist for current event
Skipping station pair 2 - 10 as doesn't exist for current event
Skipping station pair 3 - 10 as doesn't exist for current event
Skipping station pair 4 - 10 as doesn't exist for current event
Skipping station pair 5 - 10 as doesn't exist for current event
Skipping station pair 6 - 10 as doesn't exist for current event
Skipping station pair 7 - 10 as doesn't exist for current event
Skipping station pair 8 - 10 as doesn't exist for current event
Skipping station pair 9 - 10 as doesn't exist for current event
Skipping station pair 10 - 11 as doesn't exist for curren

/Users/eart0504/Documents/python/github_repositories/SeisSrcMoment/SeisSrcMoment/moment_linear_reg.py:385: RuntimeWarning: divide by zero encountered in true_divide
  linear_moment_inv_outputs[event_keys[i]]["Qs"] = 1. / event_inv_outputs[start_idx:end_idx][0::2]
/Users/eart0504/Documents/python/github_repositories/SeisSrcMoment/SeisSrcMoment/moment_linear_reg.py:387: RuntimeWarning: divide by zero encountered in true_divide
  linear_moment_inv_outputs[event_keys[i]]["Rs"] = 1. / event_inv_outputs[start_idx:end_idx][1::2]


In [ ]:
# And find corresponding moment magnitude, M_w (Hanks and Kanamori 1979):
M_w = (2./3.)*np.log10(av_M_0) - 6.0
print("Local moment magnitude, M:", M_w)

Note that this magnitude is approximately the same as that found in Alvizuri and Tape (2016), where they found that $M_W = 2.80$.

### References:

Alvizuri, C., & Tape, C. (2016). Full moment tensors for small events (Mw < 3) at Uturuncu volcano, Bolivia. Geophysical Journal International, 206(3), 1761–1783. https://doi.org/10.1093/gji/ggw247

Stork, A. L., Verdon, J. P., & Kendall, J. M. (2014). The robustness of seismic moment and magnitudes estimated using spectral analysis. Geophysical Prospecting, 62(4), 862–878. https://doi.org/10.1111/1365-2478.12134